# Yanshee_DTU

## 1. First import the robot header file

In [9]:
import YanAPI

## 2. Set the IP address of the robot to be controlled

In [10]:
import speech_recognition as sr
import requests
from playsound import playsound

In [11]:
ip_addr = "10.220.5.230" # please change to your yanshee robot IP
YanAPI.yan_api_init(ip_addr)

### RC control

In [4]:
def text_to_speech_fpt(text, api_key, voice='banmai'):
    url = 'https://api.fpt.ai/hmi/tts/v5'
    headers = {
        'api-key': api_key,
        'voice': voice,
        'speed': '',
        'format': 'mp3'
    }
    response = requests.post(url, data=text.encode('utf-8'), headers=headers)

    if response.status_code == 200:
        response_json = response.json()
        return response_json['async']
    else:
        print("Lỗi: {}, {}".format(response.status_code, response.text))
        return None

In [5]:
def download_audio_file(audio_url, save_path='/home/pi/Music/output.mp3'):
    while True:
        response = requests.get(audio_url, stream=True)

        if response.status_code == 200:
            with open(save_path, 'wb') as audio_file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        audio_file.write(chunk)
            break
        else:
            print("Lỗi khi tải xuống tệp âm thanh: {}".format(response.status_code))
    return save_path

In [6]:
def handle_command(command):
    """Parse command and provide a response based on complex commands."""
    # Define keywords
    color_keywords = {
        'màu lục': ['màu lục', 'màu nục', 'làu pục'],
        'màu lam': ['màu lam', 'làu nam', 'sàu nam']
    }
    position_keywords = {
        'bên trái': ['bên trái', 'bền trái', 'bên rái'],
        'bên phải': ['bên phải', 'bền phải', 'bên rải'],
        'ở giữa': ['ở giữa', 'ở zữa', 'ở giữ']
    }

    color = None
    position = None

    # Check for color
    for correct_color, synonyms in color_keywords.items():
        if any(keyword in command for keyword in synonyms):
            color = correct_color
            break

    # Check for position
    for correct_position, synonyms in position_keywords.items():
        if any(keyword in command for keyword in synonyms):
            position = correct_position
            break

    # Create response based on identified parts
    if color and position:
        response = "lấy vật phẩm 1 {} và đặt vào khung hộp {}.".format(color, position)
    elif color:
        response = "lấy vật phẩm 1 {}.".format(color)
    elif position:
        response = "đặt vật phẩm vào khung hộp {}.".format(position)
    else:
        response = "Tôi không hiểu lệnh này."

    return response

## 4. Phân tích dữ liệu

In [ ]:
recognizer = sr.Recognizer()
api_key = 'R3SgMgNdcDyURm119uOsqs9wEmfWg86a'  # Thay thế bằng API key của bạn

# Vòng lặp liên tục để nhận lệnh giọng nói và phản hồi
while True:
    with sr.Microphone() as source:
        print("Đang lắng nghe...")
        YanAPI.set_robot_led('button', 'red', 'on')
        audio_data = recognizer.listen(source)
        YanAPI.set_robot_led('button', 'green', 'on')

        try:
            # Nhận diện giọng nói và chuyển đổi thành văn bản
            text = recognizer.recognize_google(audio_data, language='vi-VN').strip().lower()
            print("Bạn đã nói: ", text)
            YanAPI.set_robot_led('button', 'blue', 'on')

            # Xử lý lệnh và nhận phản hồi
            command_response = handle_command(text)
            print("Phản hồi:", command_response)

            # Chuyển văn bản thành giọng nói sử dụng FPT.AI
            audio_url = text_to_speech_fpt(command_response, api_key)
            if audio_url:
                audio_path = download_audio_file(audio_url)
                if audio_path:
                    print("Tệp âm thanh đã được tải xuống: {}".format(audio_path))
                    YanAPI.delete_motion('output')
                    YanAPI.upload_media_music('/home/pi/Music/output.mp3')
                    YanAPI.start_play_music("output")
                    # os.remove(audio_path)  # Xóa tệp âm thanh sau khi phát
                else:
                    print("Không tải xuống được tệp âm thanh.")
            else:
                print("Không lấy được URL âm thanh từ API.")
                
            # Kiểm tra lệnh dừng
            if "dừng lại" in text:
                print("Dừng lại lệnh nhận diện giọng nói.")
                break

        except sr.UnknownValueError:
            print("Không nhận diện được giọng nói.")
            YanAPI.set_robot_led('button', 'blue', 'on')
            error_response = "Tôi không hiểu, hãy nói lại."
            audio_url = text_to_speech_fpt(error_response, api_key)
            if audio_url:
                audio_path = download_audio_file(audio_url)
                if audio_path:
                    try:
                        playsound(audio_path)
                    except Exception as e:
                        print("Lỗi khi phát âm thanh: {}".format(e))
        except sr.RequestError as e:
            print("Lỗi khi yêu cầu kết nối với Google Speech Recognition service: {e}".format(e))
            YanAPI.set_robot_led('button', 'blue', 'on')
        except Exception as e:
            print("Đã xảy ra lỗi: {}".format(e))